## 어드레스 자세 탐지 시 머리 초기값 저장


# 0. Install and Import Dependencies

In [211]:
#!pip install tensorflow==2.16.1 opencv-python matplotlib

In [212]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

In [213]:
# 초기 상태 저장을 위한 전역 변수 선언
initial_horizontal_change = None
initial_vertical_change = None
initial_lateral_change = None

# 1. Load Model + Draw Keypoints + Draw Edges

In [214]:
interpreter = tf.lite.Interpreter(model_path='movenet_lighting_tflite_float16.tflite')
interpreter.allocate_tensors()

In [215]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [216]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [217]:
# 어깨 중간과 엉덩이 중간에 세로선 그리기 추가
def draw_midline(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    shoulder_mid = (shaped[5][:2] + shaped[6][:2]) / 2
    hip_mid = (shaped[11][:2] + shaped[12][:2]) / 2

    if (shaped[5][2] > confidence_threshold and shaped[6][2] > confidence_threshold and
            shaped[11][2] > confidence_threshold and shaped[12][2] > confidence_threshold):
        cv2.line(frame, (int(shoulder_mid[1]), int(shoulder_mid[0])), (int(hip_mid[1]), int(hip_mid[0])), (255, 0, 0), 2)


In [218]:
def draw_face_vertical_line(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    # 머리 상단과 목 위치를 사용하여 대략적인 이마와 턱의 위치 추정
    head_top = shaped[0][:2]  # 머리 상단(이마로 가정)
    neck = shaped[1][:2]  # 목(턱 근처로 가정)

    if (shaped[0][2] > confidence_threshold and shaped[1][2] > confidence_threshold):
        # 이마에서 턱까지의 선 그리기
        cv2.line(frame, (int(head_top[1]), int(head_top[0])), (int(neck[1]), int(neck[0])), (0, 0, 255), 2)


In [219]:
def is_address_pose(keypoints_with_scores, confidence_threshold=0.4):
    keypoints = np.squeeze(keypoints_with_scores)
    
    # 키포인트 신뢰도 체크
    if (keypoints[5][2] < confidence_threshold or keypoints[6][2] < confidence_threshold or
        keypoints[11][2] < confidence_threshold or keypoints[12][2] < confidence_threshold):
        return False  # 신뢰도가 임계값 미만인 키포인트가 있으면 어드레스 자세로 판단하지 않음

    # 어깨와 골반의 중간점 계산
    shoulder_midpoint = (keypoints[5][:2] + keypoints[6][:2]) / 2
    hip_midpoint = (keypoints[11][:2] + keypoints[12][:2]) / 2

    # 어깨와 골반의 높이 차이 계산
    vertical_diff = abs(shoulder_midpoint[1] - hip_midpoint[1])
    
    # 머리-목 각도와 목-골반 각도를 계산
    head_point = keypoints[0, :2]
    neck_point = keypoints[1, :2]
    hip_center = (keypoints[11, :2] + keypoints[12, :2]) / 2
    head_neck_angle = calculate_angle(head_point, neck_point)
    neck_hip_angle = calculate_angle(neck_point, hip_center)
    
    # 각도 차이를 계산
    angle_diff = abs(head_neck_angle - neck_hip_angle)
    
    # 높이 차이와 각도 차이를 기반으로 어드레스 자세와 몸통의 직선성 판별
    vertical_diff_threshold = 5  # 높이 차이 임계값
    if vertical_diff < vertical_diff_threshold and (angle_diff < 165 and angle_diff > 145):
        return True  # 어드레스 자세이며 몸통이 직선임
    else:
        return False

In [220]:
def calculate_angle(p1, p2):
    """두 점 p1, p2 간의 각도를 계산합니다."""
    angle = np.arctan2(p2[1] - p1[1], p2[0] - p1[0]) * 180.0 / np.pi
    return angle

In [221]:
def calculate_distance(p1, p2):
    """두 점 p1, p2 사이의 유클리드 거리를 계산합니다."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [222]:
# def analyze_face_pose(keypoints):
#     nose = keypoints[0]  # 코
#     left_ear = keypoints[3]  # 왼쪽 귀
#     right_ear = keypoints[4]  # 오른쪽 귀
#     neck = keypoints[1]  # 목

#     # 수평 변화 감지: 귀와 귀 사이의 거리
#     horizontal_change = calculate_distance(left_ear[:2], right_ear[:2])
#     print(f"수평 변화 거리: {horizontal_change:.2f}")

#     # 높이 변화 감지: 코와 목 사이의 거리
#     vertical_change = calculate_distance(nose[:2], neck[:2])
#     print(f"높이 변화 거리: {vertical_change:.2f}")

#     # 좌우 거리 변화 감지: 코와 양쪽 귀의 중점 사이의 거리
#     ears_midpoint = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)
#     lateral_change = calculate_distance(nose[:2], ears_midpoint)
#     print(f"좌우 거리 변화: {lateral_change:.2f}")

In [223]:
def distance(p1, p2):
    """두 점 p1, p2 사이의 거리를 계산합니다."""
    return np.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

In [224]:
def save_initial_state_face(keypoints):
    global initial_horizontal_change, initial_vertical_change, initial_lateral_change
    # 초기 상태 계산 및 저장
    nose = keypoints[0]
    left_ear = keypoints[3]
    right_ear = keypoints[4]
    neck = keypoints[1]

    initial_horizontal_change = calculate_distance(left_ear[:2], right_ear[:2])
    initial_vertical_change = calculate_distance(nose[:2], neck[:2])
    ears_midpoint = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)
    initial_lateral_change = calculate_distance(nose[:2], ears_midpoint)

In [225]:
def analyze_swing_from_initial_face(keypoints):
    # 현재 상태 계산
    nose = keypoints[0]
    left_ear = keypoints[3]
    right_ear = keypoints[4]
    neck = keypoints[1]

    current_horizontal_change = calculate_distance(left_ear[:2], right_ear[:2])
    current_vertical_change = calculate_distance(nose[:2], neck[:2])
    ears_midpoint = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)
    current_lateral_change = calculate_distance(nose[:2], ears_midpoint)

    # 초기 상태와 비교
    horizontal_movement = current_horizontal_change - initial_horizontal_change
    vertical_movement = current_vertical_change - initial_vertical_change
    lateral_movement = current_lateral_change - initial_lateral_change

    print(f"스윙 수평 변화: {horizontal_movement:.2f}, 스윙 높이 변화: {vertical_movement:.2f}, 스윙 좌우 변화: {lateral_movement:.2f}")

In [226]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [227]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 316,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [228]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1]))

In [229]:
for edge, color in EDGES.items():
    p1, p2 = edge
    y1, x1, c1 = shaped[p1]
    y2, x2, c2 = shaped[p2]
    print((int(x2), int(y2)))

(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)


In [230]:
for kp in shaped:
    ky, kx, kp_conf = kp
    print(int(ky), int(kx), kp_conf)

0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0


# 2. Make Detections

In [231]:
cap = cv2.VideoCapture(0)

# 어드레스 자세가 감지되었는지 여부를 추적하는 플래그
address_pose_detected = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    
    # input_image = tf.cast(img, dtype=tf.float32)
    # 모델이 요구하는 입력 타입에 맞게 타입 변환
    input_image = tf.cast(img, dtype=tf.uint8)  # dtype을 tf.float32에서 tf.uint8로 변경
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    right_eye = keypoints_with_scores[0][0][2]
    left_elbow = keypoints_with_scores[0][0][7]

    # 키포인트 추출 및 조정
    keypoints = np.squeeze(np.multiply(keypoints_with_scores, [frame.shape[0], frame.shape[1], 1]))
    keypoints = keypoints[:, :2]  # x, y 좌표만 사용
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    # 기존 렌더링 코드 아래에 세로선 그리기 함수 호출 추가
    draw_midline(frame, keypoints_with_scores, 0.4)

    # # 얼굴 포즈 분석 함수 호출
    # analyze_face_pose(keypoints)

    # if is_address_pose(keypoints_with_scores, 0.4):
    #     print("어드레스 자세 감지됨!")  # 어드레스 자세와 몸통이 똑바로 서 있는 상태 감지
    #     time.sleep(1)
    # else:
    #     print("어드레스 자세 아님.")
    #     time.sleep(1)

    # 어드레스 자세 감지
    if not address_pose_detected and is_address_pose(keypoints_with_scores, 0.4):
        print("올바른 어드레스 자세 감지됨. 초기 상태를 저장합니다.")
        save_initial_state_face(keypoints)
        address_pose_detected = True
    
    # 어드레스 자세가 감지된 후 스윙 분석
    elif address_pose_detected:
        analyze_swing_from_initial_face(keypoints)
        # 추가적인 스윙 분석 로직이 필요하면 여기에 구현합니다.

    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

올바른 어드레스 자세 감지됨. 초기 상태를 저장합니다.
스윙 수평 변화: 1.54, 스윙 높이 변화: -6.92, 스윙 좌우 변화: -0.77
스윙 수평 변화: 2.77, 스윙 높이 변화: -7.45, 스윙 좌우 변화: 0.24
스윙 수평 변화: 2.24, 스윙 높이 변화: -6.55, 스윙 좌우 변화: -0.58
스윙 수평 변화: 4.60, 스윙 높이 변화: -2.82, 스윙 좌우 변화: 9.62
스윙 수평 변화: 7.27, 스윙 높이 변화: 0.04, 스윙 좌우 변화: 6.06
스윙 수평 변화: 4.28, 스윙 높이 변화: -3.14, 스윙 좌우 변화: 7.41
스윙 수평 변화: 3.89, 스윙 높이 변화: -4.04, 스윙 좌우 변화: 5.99
스윙 수평 변화: 5.26, 스윙 높이 변화: -2.89, 스윙 좌우 변화: 7.95
스윙 수평 변화: 3.59, 스윙 높이 변화: -5.83, 스윙 좌우 변화: 4.41
스윙 수평 변화: 5.44, 스윙 높이 변화: -4.85, 스윙 좌우 변화: 5.07
스윙 수평 변화: 13.08, 스윙 높이 변화: 1.19, 스윙 좌우 변화: 4.52
스윙 수평 변화: 12.21, 스윙 높이 변화: -1.48, 스윙 좌우 변화: 4.15
스윙 수평 변화: 11.68, 스윙 높이 변화: -0.20, 스윙 좌우 변화: 2.62
스윙 수평 변화: 11.92, 스윙 높이 변화: 0.09, 스윙 좌우 변화: 5.80
스윙 수평 변화: 13.12, 스윙 높이 변화: 0.72, 스윙 좌우 변화: 5.13
스윙 수평 변화: 13.54, 스윙 높이 변화: 2.24, 스윙 좌우 변화: 6.80
스윙 수평 변화: 13.09, 스윙 높이 변화: 1.73, 스윙 좌우 변화: 5.72
스윙 수평 변화: 15.36, 스윙 높이 변화: 1.38, 스윙 좌우 변화: 3.27
스윙 수평 변화: 20.38, 스윙 높이 변화: -0.29, 스윙 좌우 변화: 12.28
스윙 수평 변화: 22.22, 스윙 높이 변화: 1.46, 스윙 좌우 변화: 14.25
스윙 